In [1]:
import torch
from torch import tensor, cat, arange, empty, rand, inference_mode, Tensor
from torch.nn import Module, Linear, ReLU, Dropout, ModuleList, BCEWithLogitsLoss
from torch.nn.functional import binary_cross_entropy_with_logits
from torch.utils.data import DataLoader, Dataset, TensorDataset
from torch.optim import AdamW
from sklearn.linear_model import LogisticRegression
from more_itertools import pairwise
from math import lcm
from modint import chinese_remainder
import random
from tqdm import tqdm
import pickle
from plotly.graph_objects import Figure, Scatter, Heatmap, Layout
from plotly.express.colors import DEFAULT_PLOTLY_COLORS
from statistics import mean
from typing import Iterable, List, Tuple, Dict
from itertools import combinations
from dataclasses import dataclass

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("using", "device")

using device


In [3]:
def random_divisible(bound, divisors, non_divisors):
    while True:
        modulus = lcm(*divisors, *non_divisors)
        remainders = [0 for _ in divisors] + [random.randrange(1, d) for d in non_divisors]
        remainder = chinese_remainder(divisors + non_divisors, remainders)
        n = random.randrange(0, bound, modulus) + remainder
        if n in range(bound):
            return n

def bits(n, nbits):
    return [(n // 2**i) % 2 for i in range(nbits)]

def from_bits(bits: Tensor):
    return (bits * 2 ** arange(bits.size(-1))).sum(-1)

@inference_mode()
def make_divisibility_dataset(size, nbits, divisors, labeled_divisors, tqdm_=True):
    nums = empty(size, nbits, dtype=torch.bool)
    divisibilities = empty(size, len(labeled_divisors), dtype=torch.bool)

    for i in tqdm(range(size)) if tqdm_ else range(size):
        divisibility = rand(len(divisors)) >= 0.5
        n = random_divisible( 2**nbits,
                              divisors     = [divisor for i, divisor in enumerate(divisors) if     divisibility[i]],
                              non_divisors = [divisor for i, divisor in enumerate(divisors) if not divisibility[i]] )
        divisibilities[i, :] = tensor([n % d == 0 for d in labeled_divisors])
        nums[i, :] = tensor(bits(n, nbits))

    return TensorDataset(nums, divisibilities)

def bits_accuracy_fn(pred, y):
    assert pred.is_floating_point()
    assert y.dtype == torch.bool
    return ((pred >= 0) == y).all(-1).float().mean()

In [4]:
class MLP(Module):
    def __init__(self, layer_sizes, dropout, activation_function=ReLU()):
        super().__init__()
        self.linears = ModuleList(Linear(size_in, size_out) for size_in, size_out in pairwise(layer_sizes))
        self.activation_function = activation_function
        self.dropout = Dropout(dropout)

    def forward(self, x, return_activations=False):
        activations = []
        for linear in self.linears[:-1]:
            if return_activations:
                activations.append(x)
            x = linear(x)
            x = self.activation_function(x)
            x = self.dropout(x)
        if return_activations:
            activations.append(x)
        x = self.linears[-1](x)
        if return_activations:
            activations.append(x)
        return (x, activations) if return_activations else x

In [5]:
@dataclass(frozen=True)
class ModelType:
    dropout: float
    layer_sizes: Tuple

@dataclass(frozen=True)
class ProbePosition:
    model_type: ModelType
    layer: int
    label: int

In [6]:
def train( model,
           dataloader,
           epochs,
           lr=1e-3,
           loss_fn=BCEWithLogitsLoss(),
           epoch_tqdm=True,
           dataloader_tqdm=False,
           tqdm_desc="",
           plot_loss=True ):
    
    if tqdm_desc is not None:
        tqdm_desc = f"{tqdm_desc} "

    model.train()
    optimizer = AdamW(model.parameters(), lr=lr)
    loss_history = []
    for epoch in tqdm(range(epochs), desc=tqdm_desc) if epoch_tqdm else range(epochs):
        epoch_loss_history = []
        for x, y in tqdm(dataloader, desc=f"{tqdm_desc}epoch {epoch+1}/{epochs}") if dataloader_tqdm else dataloader:
            loss = loss_fn(model(x.float()), y.float())
            epoch_loss_history.append(loss.item())
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        loss_history.append(mean(epoch_loss_history))

    if plot_loss:
        display(Figure( Scatter(y=loss_history),
                        layout=Layout(xaxis_title="Epoch.", yaxis_title="Loss.", title="Training loss history.") ))

@dataclass
class TestResults:
    loss: float
    accuracy: float

@inference_mode()
def test(model, dataloader, loss_fn=BCEWithLogitsLoss(), accuracy_fn=bits_accuracy_fn, tqdm_=True):
    model.eval() # we want to test the model with dropout
    losses = []
    accuracies = []
    for x, y in tqdm(dataloader) if tqdm_ else dataloader:
        pred = model(x.float())
        losses.append(loss_fn(pred, y.float()).item())
        accuracies.append(accuracy_fn(pred, y).item())
    return TestResults(loss=mean(losses), accuracy=mean(accuracies))

In [16]:
NBITS = 16
DATASET_DIVISORS = [3, 5, 7, 11]
DATASET_LABELED_DIVISORS = [3*11, 5*7]
PROBED_DIVISORS = [3, 5, 7, 11]
PROBED_DIVISOR_PAIR_INDICES = list(combinations(range(len(PROBED_DIVISORS)), 2)) # [(0, 1), (0, 2), (1, 3), (2, 3)]

In [8]:
train_dataset = make_divisibility_dataset(size=2**15, nbits=NBITS, divisors=DATASET_DIVISORS, labeled_divisors=DATASET_LABELED_DIVISORS)
test_dataset  = make_divisibility_dataset(size=2**10, nbits=NBITS, divisors=DATASET_DIVISORS, labeled_divisors=DATASET_LABELED_DIVISORS)
train_dataloader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_dataloader  = DataLoader(test_dataset,  batch_size=128)

models = { ModelType(dropout=dropout, layer_sizes=tuple(layer_sizes)): MLP(layer_sizes, dropout=dropout)
           for dropout in [0.0, 0.01, 0.02, 0.05, 0.075, 0.1, 0.125, 0.15, 0.175, 0.2, 0.25]
           for layer_sizes in [ [NBITS, 4*NBITS, len(DATASET_LABELED_DIVISORS)],
                                [NBITS, 4*NBITS, 4*NBITS, len(DATASET_LABELED_DIVISORS)],
                                [NBITS, 8*NBITS, 8*NBITS, 8*NBITS, len(DATASET_LABELED_DIVISORS)] ] }

for model_type, model in models.items():
    train(model, train_dataloader, epochs=250, lr=1e-3, tqdm_desc=model_type)

test_results = { model_type: test(model, test_dataloader)
                 for model_type, model in models.items() }

ModelType(dropout=0.0, layer_sizes=(16, 64, 2)) : 100%|██████████| 250/250 [01:37<00:00,  2.57it/s]


ModelType(dropout=0.0, layer_sizes=(16, 64, 64, 2)) : 100%|██████████| 250/250 [01:57<00:00,  2.14it/s]


ModelType(dropout=0.0, layer_sizes=(16, 128, 128, 128, 2)) : 100%|██████████| 250/250 [02:42<00:00,  1.54it/s]


ModelType(dropout=0.01, layer_sizes=(16, 64, 2)) : 100%|██████████| 250/250 [01:40<00:00,  2.48it/s]


ModelType(dropout=0.01, layer_sizes=(16, 64, 64, 2)) : 100%|██████████| 250/250 [02:11<00:00,  1.90it/s]


...

ModelType(dropout=0.2, layer_sizes=(16, 128, 128, 128, 2)) : 100%|██████████| 250/250 [03:09<00:00,  1.32it/s]


ModelType(dropout=0.25, layer_sizes=(16, 64, 2)) : 100%|██████████| 250/250 [02:01<00:00,  2.05it/s]


ModelType(dropout=0.25, layer_sizes=(16, 64, 64, 2)) : 100%|██████████| 250/250 [02:16<00:00,  1.83it/s]


ModelType(dropout=0.25, layer_sizes=(16, 128, 128, 128, 2)) : 100%|██████████| 250/250 [03:04<00:00,  1.36it/s]


100%|██████████| 8/8 [00:00<00:00, 647.53it/s]


In [9]:
with open("divisibility-16-bits.pickle", "wb") as f:
    pickle.dump(models, f)

In [10]:
def plot_test_accuracies(test_results: Dict[ModelType, TestResults]):
    model_types         = list(set(test_results.keys()))
    all_layer_sizes     = list(set(model_type.layer_sizes for model_type in model_types))
    all_dropouts        = sorted(list(set(model_type.dropout for model_type in model_types)))
    all_layer_sizes_str = [", ".join(map(str, layer_sizes)) for layer_sizes in all_layer_sizes]
    all_dropouts_str    = [str(dropout) for dropout in all_dropouts]

    accuracies = [ [ test_results[ModelType(dropout=dropout, layer_sizes=layer_sizes)].accuracy
                     for dropout in all_dropouts ]
                   for layer_sizes in all_layer_sizes ]
    
    accuracies_str = [[f"{accuracy:.2%}" for accuracy in row] for row in accuracies]
  
    fig = Figure( Heatmap( x    = all_dropouts_str,
                           y    = all_layer_sizes_str,
                           z    = accuracies,
                           text = accuracies_str,
                           zmin = 0.0,
                           zmax = 1.0,
                           texttemplate="%{text}" ),
                  layout=Layout( title       = "Model test accuracies.",
                                 xaxis_title = "Dropout.",
                                 yaxis_title = "Layer sizes." ) )

    display(fig)


plot_test_accuracies(test_results)

In [11]:
def allequal(xs):
    return all(current == next for current, next in pairwise(xs))

@dataclass
class ActivationsAndLabels:
    activations: List[Tensor]
    labels: Tensor
    label_descriptions: List[str]

class ActivationsAndLabelsDataset(Dataset):
    def __init__(self, activations: Iterable[Tensor], labels: Tensor, label_descriptions: Iterable[str]):
        self.activations = list(activations)
        self.labels = labels
        self.nlabels = self.labels.size(-1)
        self.label_descriptions = list(label_descriptions)

        assert allequal([act.size(0) for act in self.activations] + [self.labels.size(0)])
        assert len(self.label_descriptions) == self.labels.size(-1)

    def __getitem__(self, index) -> ActivationsAndLabels:
        return ActivationsAndLabels( activations        = [act[index, ...] for act in self.activations],
                                     labels             = self.labels[index, ...],
                                     label_descriptions = self.label_descriptions )

@inference_mode()
def collect_activations(model, dataset):
    model.eval()
    data, _ = dataset[:]
    data = data.float()
    _, activations = model(data, return_activations=True)
    return activations

@inference_mode()
def divisibility_labels(dataset, divisors):
    bits, _ = dataset[:]
    nums = from_bits(bits).unsqueeze(1)
    labels = cat(tuple(nums % divisor == 0 for divisor in divisors), dim=1)
    label_descriptions = [f"divisible by {divisor}" for divisor in divisors]
    return labels, label_descriptions

def make_divisibility_probing_dataset(model, dataset, divisors):
    activations = collect_activations(model, dataset)
    labels, label_descriptions = divisibility_labels(dataset, divisors)
    return ActivationsAndLabelsDataset(activations=activations, labels=labels, label_descriptions=label_descriptions)

@inference_mode
def xor_list_of_tensors(tensors: List[Tensor]):
        x = tensors[0]
        for y in tensors[1:]:
            x = x.logical_xor(y)
        return x

def xored_labels(dataset: ActivationsAndLabelsDataset, xored_label_indices: Iterable[Tuple[int]]):
    if not isinstance(xored_label_indices, list):
        xored_label_indices = list(xored_label_indices)

    xored_labels             = [ xor_list_of_tensors([dataset.labels[:, i] for i in label_indices])
                                 for label_indices in xored_label_indices ]
    xored_label_descriptions = [ " xor ".join(dataset.label_descriptions[i] for i in label_indices)
                                 for label_indices in xored_label_indices ]
    xored_labels = cat( tuple(label.unsqueeze(-1) for label in xored_labels),
                        dim=-1 )
    return ActivationsAndLabelsDataset(activations=dataset.activations, labels=xored_labels, label_descriptions=xored_label_descriptions)

def xored_label_pairs(dataset: ActivationsAndLabelsDataset):
    return xored_labels(dataset, xored_label_indices=combinations(range(dataset.nlabels), 2))

In [12]:
def train_probe(data: Tensor, labels: Tensor) -> Linear:
    sklearn_probe = LogisticRegression(class_weight="balanced", solver="newton-cholesky")
    sklearn_probe.fit(data, labels)
    
    torch_probe = Linear(data.shape[-1], 1)
    torch_probe.weight.data.copy_(tensor(sklearn_probe.coef_))
    torch_probe.bias  .data.copy_(tensor(sklearn_probe.intercept_))

    return torch_probe

@inference_mode()
def test_probe(probe: Linear, data: Tensor, labels: Tensor) -> TestResults:
    assert labels.dtype == torch.bool

    pred = probe(data).squeeze(-1)

    return TestResults( loss     = binary_cross_entropy_with_logits(pred, labels.float()).item(),
                        accuracy = ((pred >= 0) == labels).float().mean().item() )

def train_probes( datasets: Dict[ModelType, ActivationsAndLabelsDataset],
                  tqdm_=True,
                  tqdm_desc=None ) -> Dict[ProbePosition, Linear]:
    probes = dict()

    positions = [ ((model_type, dataset), label, layer) for model_type, dataset in datasets.items()
                                                        for layer in range(len(dataset[:0].activations))
                                                        for label in range(dataset[:0].labels.size(-1)) ]
    
    random.shuffle(positions) # training probes for some model types, labels, or layers takes disproportionately more time
                              # than for others. shuffle positions to avoid surprises like the last 10% of the tqdm taking
                              # 10 times longer than the first 90% because it so turned out training probes for models with
                              # bigger dropout takes longer
    
    for (model_type, dataset), label, layer in tqdm(positions, desc=tqdm_desc) if tqdm_ else positions:
        data = dataset[:]
        probe = train_probe(data.activations[layer], data.labels[:, label])
        probe_position = ProbePosition(model_type=model_type, layer=layer, label=label)
        probes[probe_position] = probe
    
    return probes

def test_probes( probes: Dict[ProbePosition, Linear],
                 datasets: Dict[ModelType, ActivationsAndLabelsDataset],
                 tqdm_=True,
                 tqdm_desc=None ) -> Dict[ProbePosition, TestResults]:

    test_results = dict()
    for probe_position, probe in tqdm(probes.items(), desc=tqdm_desc) if tqdm_ else probes.items():
        data = datasets[probe_position.model_type][:]
        results = test_probe( probe,
                              data   = data.activations[probe_position.layer],
                              labels = data.labels[:, probe_position.label] )
        test_results[probe_position] = results
        
    return test_results

def plot_probe_accuracies( label_probe_test_results: Dict[ProbePosition, TestResults],
                           xored_label_probe_test_results: Dict[ProbePosition, TestResults],
                           label_descriptions,
                           xored_label_descriptions ):
    
    label_keys       = list(label_probe_test_results.keys())
    xored_label_keys = list(xored_label_probe_test_results.keys())
    all_keys         = label_keys + xored_label_keys
    all_layer_sizes  = sorted(list(set(probe_position.model_type.layer_sizes for probe_position in all_keys)))
    dropouts         = sorted(list(set(probe_position.model_type.dropout     for probe_position in all_keys)))
    labels           = sorted(list(set(probe_position.label                  for probe_position in label_keys)))
    xored_labels     = sorted(list(set(probe_position.label                  for probe_position in xored_label_keys)))
    
    for layer_sizes in all_layer_sizes:
        for layer in range(len(layer_sizes)):
            fig = Figure(layout=Layout( title       = f"Probe accuracies on layer {layer} of model with"
                                                      f"layer sizes {', '.join(map(str, layer_sizes))}.",
                                        xaxis_title = "Dropout while trianing model.",
                                        yaxis_title = "Probe accuracy.", 
                                        yaxis       = {"range": [0.5, 1.0]} ))
            
            accuracies = dict()
            for label in labels:
                label_description = label_descriptions[label]
                accuracies[label_description] = []
                for dropout in dropouts:
                    probe_position = ProbePosition( model_type=ModelType(dropout=dropout, layer_sizes=layer_sizes),
                                                    layer=layer,
                                                    label=label )
                    accuracy = label_probe_test_results[probe_position].accuracy
                    accuracies[label_description].append(accuracy)

            xored_accuracies = dict()
            for label in xored_labels:
                label_description = xored_label_descriptions[label]
                xored_accuracies[label_description] = []
                for dropout in dropouts:
                    probe_position = ProbePosition( model_type=ModelType(dropout=dropout, layer_sizes=layer_sizes),
                                                    layer=layer,
                                                    label=label )
                    accuracy = xored_label_probe_test_results[probe_position].accuracy
                    xored_accuracies[label_description].append(accuracy)

            for label_description in label_descriptions:
                fig.add_trace(Scatter( x    = dropouts,
                                       y    = accuracies[label_description],
                                       name = label_description ))
            
            for label_description in xored_label_descriptions:
                fig.add_trace(Scatter( x    = dropouts,
                                       y    = xored_accuracies[label_description],
                                       name = label_description,
                                       line = {"dash": "dot"} ))
            display(fig)

In [18]:
divisibility_probing_datasets = { model_type: make_divisibility_probing_dataset(model, train_dataset, divisors=PROBED_DIVISORS)
                                  for model_type, model in tqdm(models.items(), desc="making probing datasets.") }

xor_divisibility_probing_dataset = { model_type: xored_labels(dataset, PROBED_DIVISOR_PAIR_INDICES)
                                     for model_type, dataset in tqdm(divisibility_probing_datasets.items(), desc="xoring labels.") }

divisibility_probes     = train_probes(divisibility_probing_datasets,    tqdm_desc="training probes")
xor_divisibility_probes = train_probes(xor_divisibility_probing_dataset, tqdm_desc="training xor probes")

divisibility_probe_test_results      = test_probes( divisibility_probes,
                                                    divisibility_probing_datasets,
                                                    tqdm_desc="testing divisibility_probes" )
xor_divisibility_probes_test_results = test_probes( xor_divisibility_probes,
                                                    xor_divisibility_probing_dataset,
                                                    tqdm_desc="testing xor divisibility probes" )

plot_probe_accuracies( divisibility_probe_test_results,
                       xor_divisibility_probes_test_results,
                       label_descriptions=next(iter(divisibility_probing_datasets.values())).label_descriptions,
                       xored_label_descriptions=next(iter(xor_divisibility_probing_dataset.values())).label_descriptions )













making probing datasets.: 100%|██████████| 33/33 [00:00<00:00, 47.98it/s]


xoring labels.: 100%|██████████| 33/33 [00:00<00:00, 1551.46it/s]


































































































































































































































































































































































































































































































































































































































































































training probes: 100%|██████████| 528/528 [01:19<00:00,  6.62it/s]









































































































